# 은평구 (1101)
- 작성자 : 황다은
- 작성일자 : 2020년 11월 1일

In [22]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [23]:
df = pd.read_csv("201019one_shot.csv")
soup = BeautifulSoup(df['html'][1], 'html.parser')

### 2. 간단한 전처리 하기

In [243]:
routelen = len(soup.select('#a14_group')[0].select('tr.swtr'))
tr = soup.select('#a14_group')[0].select('tbody > tr')
trlen = len(tr)

### 3. 저장할 DataFrame 미리 만들기

In [267]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt','age','sex']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [268]:
for i in tqdm(range(routelen)):
    number = re.findall("\d*", tr[2*i].select('a')[0].text)[0]
    num = '1101' + '0' * (5- len(str(number))) + str(number)
    rsd = tr[2*i].select('td')[0].text
    cnd = tr[2*i].select('td')[1].text
    atn = tr[2*i].select('td')[2].text
    syd = tr[2*i+1].select('td')[0].text
    if '증상일' not in syd:
        syy = 0; syd = ''
    else:
        syy = 1; syd = re.sub('[증상일\(\)]', '', re.findall("증상일\(\d.+\)", syd)[0])
    
    pth = ''; dch = ''; adt = ''; age = ''; sex = '';
    
    new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
    patient = pd.concat([patient, new_pat])

    table = tr[2*i+1].select('table')[0]
    new_route = pd.read_html(str(table))[0]
    new_route.columns = ['rgl','rgt','frm','exd','ste']
    new_route['num'] = num
    new_route['ord'] = range(1, len(new_route)+1)
    new_route['dat'] = dat
    new_route['msk'] = ''
    new_route['mob'] = ''
    new_route = new_route.loc[:,rou_col]

    route = pd.concat([route, new_route])
    
for i in range(routelen*2, trlen):
    try:
        number = re.findall("\d*", tr[i].select('a')[0].text)[0]
        num = '1101' + '0' * (5- len(str(number))) + str(number)
        rsd = tr[i].select('td')[0].text
        cnd = tr[i].select('td')[1].text
        atn = tr[i].select('td')[2].text
    
        syy = ''; syd = ''; pth = ''; dch = ''; adt = ''; age = ''; sex = '';
    
        new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
        patient = pd.concat([patient, new_pat])
            
        new_route = pd.DataFrame(columns = rou_col, data = [[num, 1, dat, '', '', '', '', '', '', '']])
        route = pd.concat([route, new_route])
    except:
        continue


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 36.46it/s]


### 5. 결과 잘 나왔나 확인하기

In [269]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt,age,sex
0,110100261,2020-11-02,,1,10-16,2020.10.18.,녹번동,적십자병원,,,,
0,110100260,2020-11-02,,1,10-05,2020.10.16.,역촌동,태릉생활치료센터,,,,
0,110100259,2020-11-02,,1,10-13,2020.10.15.,갈현2동,남산생활치료센터,,,,
0,110100258,2020-11-02,,1,10-12,2020.10.14.,증산동,은평생활치료센터,,,,
0,110100257,2020-11-02,,1,10-13,2020.10.14.,녹번동,보라매병원,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
0,110100005,2020-11-02,,,,2020.2.26.,진관동,보라매병원\n 퇴원,,,,
0,110100004,2020-11-02,,,,2020.2.26.,진관동,보라매병원\n 퇴원,,,,
0,110100003,2020-11-02,,,,2020.2.25.,대조동,서북병원\n 퇴원,,,,
0,110100002,2020-11-02,,,,2020.2.25.,역촌동,서울보라매병원\n 퇴원,,,,


In [270]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,110100261,1,2020-11-02,녹번동,자택,NaN,자가격리중으로 별도 이동동선 없음,,소독완료,
0,110100260,1,2020-11-02,역촌동,마트,**마트 (접촉자 분류 완료),20.10.3.(토) 18:57 20.10.4.(일) 17:37 20.10.9....,,소독완료 (마스크착용),
1,110100260,2,2020-11-02,응암1동,편의점,**편의점 (접촉자 분류 완료),20.10.3.(토) 21:44,,소독완료 (마스크착용),
2,110100260,3,2020-11-02,응암1동,음식점,**식당 (접촉자 분류 완료),20.10.3.(토) 23:37,,소독완료 (식음외 마스크착용),
3,110100260,4,2020-11-02,신사1동,음식점,**식당 (접촉자 분류 완료),20.10.3.(토) 23:54,,"소독완료 (포장주문, 마스크착용)",
...,...,...,...,...,...,...,...,...,...,...
0,110100005,1,2020-11-02,,,,,,,
0,110100004,1,2020-11-02,,,,,,,
0,110100003,1,2020-11-02,,,,,,,
0,110100002,1,2020-11-02,,,,,,,
